In [40]:
#경고무시
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from functools import reduce
from glob import glob

pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

%matplotlib inline

# 독립변수 전처리
- 인구데이터 : 연령별 인구수
- 가구데이터 : 주택종류별 가구수
- 주택데이터 : 주택종류별, 연면적별 주택수

In [41]:
#인구, 가구, 주택 데이터 불러오기
pop = pd.read_excel(r'C:\Users\user\Desktop\인턴\0.분석과제\2.학생수예측\상관분석\5개년\1.전처리완/인구.xlsx',thousands = ',')
household = pd.read_excel(r'C:\Users\user\Desktop\인턴\0.분석과제\2.학생수예측\상관분석\5개년\1.전처리완/가구.xlsx',thousands = ',')
house = pd.read_excel(r'C:\Users\user\Desktop\인턴\0.분석과제\2.학생수예측\상관분석\5개년\1.전처리완/주택.xlsx',thousands = ',')

In [42]:
pop.head()

,시군,연령별,2016_총인구(명),2017_총인구(명),2018_총인구(명),2019_총인구(명),2020_총인구(명)
0,수원시,0세,11590,9965,9206,8161,7233
1,수원시,1세,12128,11449,9810,8889,7977
2,수원시,2세,11806,12026,11118,9527,8587
3,수원시,3세,11778,11571,11801,10832,9150
4,수원시,4세,12571,11579,11343,11523,10468


In [43]:
household.head()

,시군,주택종류,2016_일반가구(가구),2017_일반가구(가구),2018_일반가구(가구),2019_일반가구(가구),2020_일반가구(가구)
0,수원시,계,426881,434429,439323,446650,454866
1,수원시,단독주택,121983,120202,112268,109146,109686
2,수원시,아파트,239137,243599,253807,260508,263430
3,수원시,연립주택,7011,7678,7581,7736,7954
4,수원시,다세대주택,43190,45559,46320,47706,49617


In [44]:
house.head()

,시군,주택종류,연면적별,2016_주택_계,2017_주택_계,2018_주택_계,2019_주택_계,2020_주택_계
0,수원시,단독주택,20㎡ 이하,152,144,128,95,84
1,수원시,단독주택,20㎡ ~ 40㎡,1238,1185,1117,935,845
2,수원시,단독주택,40㎡ ~ 60㎡,2107,2044,1969,1709,1591
3,수원시,단독주택,60㎡ ~ 85㎡,3323,3222,3099,2793,2583
4,수원시,단독주택,85㎡ ~ 100㎡,1968,1936,1878,1732,1620


In [45]:
# 문자열 컬럼 공백 제거
pop['시군'] = pop['시군'].map(lambda x : x.strip())
pop['연령별'] = pop['연령별'].map(lambda x : x.strip())
household['시군'] = household['시군'].map(lambda x : x.strip())
household['주택종류'] = household['주택종류'].map(lambda x : x.strip())
house['시군'] = house['시군'].map(lambda x : x.strip())
house['주택종류'] = house['주택종류'].map(lambda x : x.strip())

In [46]:
#각 데이터 범주에 데이터 출처 표시
pop['연령별'] = pop['연령별'].apply(lambda x : x+'(인구수)')
household['주택종류'] = household['주택종류'].apply(lambda x : x+'(가구수)')
house['연면적별'] = house['연면적별'].apply(lambda x : x+'(주택수)')

In [47]:
# 연도별 데이터프레임 생성
for year in [2016,2017,2018,2019,2020]:    
    #인구데이터 : 연령범주 컬럼화
    pop_ = pd.crosstab(index = pop['시군'], columns = pop['연령별'],values = pop['{}_총인구(명)'.format(year)],aggfunc = np.sum).reset_index()
    pop_.columns.name = None

    #연령 범주 컬럼 생성
    pop_['만3-5세(인구수)'] = pop_['3세(인구수)'] + pop_['4세(인구수)'] + pop_['5세(인구수)']
    pop_['만6-11세(인구수)'] = pop_['6세(인구수)'] + pop_['7세(인구수)'] + pop_['8세(인구수)'] + pop_['9세(인구수)'] + pop_['10세(인구수)'] + pop_['11세(인구수)']
    pop_['만12-14세(인구수)'] = pop_['12세(인구수)'] + pop_['13세(인구수)'] + pop_['14세(인구수)']
    pop_['만15-17세(인구수)'] = pop_['15세(인구수)'] + pop_['16세(인구수)'] + pop_['17세(인구수)']

    #단일연령컬럼 제거
    pop_.drop(['{}세(인구수)'.format(i) for i in range(0,20)],axis=1,inplace = True)

    #가구데이터 : 주택종류 범주 컬럼화
    household_ = pd.crosstab(index = household['시군'],columns = household['주택종류'],values = household['{}_일반가구(가구)'.format(year)],aggfunc = np.sum).reset_index()
    household_.columns.name = None

    #주택 데이터 : 주택종류, 연면전별 범주 컬럼화
    house_ = pd.crosstab(index = house['시군'],columns = [house['주택종류'],house['연면적별']],values = house['{}_주택_계'.format(year)],aggfunc = np.sum).reset_index()
    house_.columns.name = None

    #멀티컬럼 단일컬럼화 
    temp_df = pd.DataFrame()
    for col in house_.columns:
        col_name = [col[0] if col[0] == '시군' else col[0]+'_'+col[1]][0]
        temp_df[col_name] = house_[col]
    house_ = temp_df

    pre_cols = [col for col in house_.columns[1:] if '60㎡ ~ 85㎡' not in col]
    temp_df = pd.DataFrame()
    temp_df['시군'] = house_['시군']
    for cat in house['주택종류'].unique() : 
        temp_df['{}_60㎡미만(주택수)'.format(cat)] = house_['{}_20㎡ 이하(주택수)'.format(cat)]  + house_['{}_20㎡ ~ 40㎡(주택수)'.format(cat)] + house_['{}_40㎡ ~ 60㎡(주택수)'.format(cat)]
        temp_df['{}_60㎡ ~ 85㎡(주택수)'.format(cat)] = house_['{}_60㎡ ~ 85㎡(주택수)'.format(cat)]
        temp_df['{}_85㎡이상(주택수)'.format(cat)] = house_['{}_85㎡ ~ 100㎡(주택수)'.format(cat)]  + house_['{}_100㎡ ~ 130㎡(주택수)'.format(cat)] + house_['{}_130㎡ ~ 165㎡(주택수)'.format(cat)] + house_['{}_165㎡ ~ 230㎡(주택수)'.format(cat)]  + house_['{}_230㎡ 초과(주택수)'.format(cat)]

    house_ = temp_df

    #데이터프레임 병합
    globals()['df_{}'.format(year)] = reduce(lambda left,right : pd.merge(left,right,on='시군',how = 'left'),[pop_,household_,house_])

    #'계' 포함 컬럼 제거
    globals()['df_{}'.format(year)].drop([col for col in globals()['df_{}'.format(year)].columns if '계' in col],axis=1, inplace = True)
    
    #'년도' 컬럼 생성
    globals()['df_{}'.format(year)]['년도']= year
    
    #컬럼 순서 변경('년도' 맨 앞에)
    except_year_col = globals()['df_{}'.format(year)].columns.tolist()
    except_year_col.remove('년도')
    globals()['df_{}'.format(year)] = globals()['df_{}'.format(year)][['년도']+except_year_col]

In [48]:
#년도별 데이터프레임 병합
df = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020],axis=0)

In [49]:
# # 독립변수 데이터프레임 저장
# df.to_csv('C:/Users/user/Desktop/상관분석_5개년.csv',encoding='cp949',index= False)